In [2]:
import pandas as pd
import nltk
from gensim.models import Word2Vec, LsiModel, KeyedVectors
from scipy.spatial import distance
import pymorphy2

In [2]:
"""""""""
- степень тематической связи аббревиатуры и определения (LSA)
"""""""""

'\n- степень почти-синонимии аббревиатуры и определения (word2vec)\n- степень тематической связи аббревиатуры и определения (LSA)\n- степень схожести поверхности слов (LCS)\n'

In [3]:
def compare_sentences(s1, s2):
    s1 = nltk.word_tokenize(s1)
    s2 = nltk.word_tokenize(s2)
    if len(s1)!=len(s2):
        return False
    morph = pymorphy2.MorphAnalyzer()
    equal = True
    for i in range(len(s1)):
        w1 = morph.parse(s1[i])[0]
        norm1 = w1.normal_form
        w2 = morph.parse(s2[i])[0]
        norm2 = w2.normal_form
        equal = equal and norm1==norm2
    return equal

In [93]:
def is_it_correct(df, correct_pairs_df):
    print(len(df))
    is_correct = []
    for i in range(len(df)):
        A = df['abbreviation'][i].lower()
        D = df['definition'][i].lower()
        l = 0
        r = len(correct_pairs_df)
        while(l < r-1):
            m = (l+r)//2
            #print('l = ', l, ' r = ', r, ' m = ', m)
            if correct_pairs_df['abbreviation'][m].lower() <= A:
                l = m
            else:
                r = m
        #print('done')
        if (correct_pairs_df['abbreviation'][l].lower()==A and
            compare_sentences(correct_pairs_df['definition'][l].lower(), D)):
            is_correct.append(1)
        else:
            is_correct.append(0)
        #print('i = ', i)
    return is_correct

In [5]:
#неприрывный числовой атрибут
def common_first_letters_cnt(df):
    feature = []
    for i in range(len(df)):
        A = df['abbreviation'][i]
        D = nltk.word_tokenize(df['definition'][i])
        common_letters = 0
        for word in D:
            if word.lower()[0] in A.lower():
                common_letters += 1
        feature.append(common_letters/len(A))
    return feature

In [6]:
#номинальный категоральный атрибут
def parenthesis_feature(df, pairs):
    feature = []
    for i in range(len(pairs)):
        abbreviation_in_parenthesis = False
        definition_in_parenthesis = False
        idx = pairs['abbreviation_place'][i]
        l = pairs['begin'][i]
        r = pairs['end'][i]
        left_p = False
        right_p = False
        for j in range(max(0, idx-5), min(len(df), idx+6)):
            if df['token'][j]=='(':
                left_p = True
            if df['token'][j]==')' and left_p:
                right_p = True
        abbreviation_in_parenthesis = left_p and right_p
        
        left_p = False
        right_p = False
        for j in range(max(0, l-5), l):
            if df['token'][j]=='(':
                left_p = True
        for j in range(r+1, min(len(df), r+6)):
            if df['token'][j]==')':
                right_p = True
        definition_in_parenthesis = left_p and right_p
        
        feature.append(int(abbreviation_in_parenthesis or definition_in_parenthesis))
    return feature

In [7]:
def add_vectors(v, w):
    #print('add')
    res = [vi + wi for vi, wi in zip(v, w)]
    return res

In [8]:
def divide_vector(vec, div):
    res = [el/div for el in vec]
    return res

In [23]:
#неприрывный числовой атрибут
def almost_synonyms(pairs):
    morph = pymorphy2.MorphAnalyzer()
    model = KeyedVectors.load_word2vec_format(
        '../dataset/fit_language_models/word2vec_model.bin', binary=True)
    feature = []
    for i in range(len(pairs)):
        A = pairs['abbreviation'][i].lower()
        D = nltk.word_tokenize(pairs['definition'][i].lower())
        try:
            omega = model[A]
            eta = [0]*len(omega)
            l = 0
            for word in D:
                #print(1)
                try:
                    w = morph.parse(word)[0]
                    norm = w.normal_form
                    vec = model[norm]
                    eta = add_vectors(eta, vec)
                    #print(2.1)
                    l += 1
                except KeyError: 
                    continue
            eta = divide_vector(eta, l)
            dis = distance.cosine(omega, eta)
            feature.append(dis)
        except KeyError:
            #print(A)
            feature.append(None)
        except ZeroDivisionError:
            feature.append(None)
    return feature

In [64]:
def longest_common_subsequence(A, D):
    D = nltk.word_tokenize(D)
    D = ''.join(D)
    #print(A)
    #print(D)
    n = len(A)
    m = len(D)
    matrix = [["" for x in range(m)] for x in range(n)]
    for i in range(n):
        for j in range(m):
            if A[i]==D[j]:
                if i==0 or j==0:
                    matrix[i][j] = A[i]
                else:
                    matrix[i][j] = matrix[i-1][j-1] + A[i]
            else:
                matrix[i][j] = max(matrix[i-1][j], matrix[i][j-1], key=len)
    cs = matrix[-1][-1]
    #print(len(cs))
    return len(cs)/n

In [65]:
#неприрывный числовой атрибут
def lcs_feature(df):
    feature = []
    for i in range(len(df)):
        A = df['abbreviation'][i]
        D = df['definition'][i]
        lcs = longest_common_subsequence(A.lower(), D.lower())
        feature.append(lcs)
    return feature

In [68]:
df.head()

,Unnamed: 0,token,is_abbreviation
0,0,Недостаточность,0
1,1,дефицит,0
2,2,витамина,0
3,3,D,0
4,4,женщин,0


In [69]:
pairs.head()

,Unnamed: 0,abbreviation,definition,distance,begin,end,abbreviation_place
0,0,РФ,регуляции репродуктивной функции,14,20,23,36
1,1,РФ,регуляции репродуктивной функции .,13,20,24,36
2,2,РФ,репродуктивной функции,14,21,23,36
3,3,РФ,репродуктивной функции .,13,21,24,36
4,4,РФ,репродуктивной функции . Целью,12,21,25,36


In [94]:
pairs = pd.read_csv("../dataset/ready_data/potential_pairs.csv")
df = pd.read_excel("../dataset/ready_data/merges_data.xlsx")
correct_pairs = pd.read_excel("../dataset/ready_data/merged_data_correct_pairs.xlsx")
pairs['is_it_correct'] = is_it_correct(pairs, correct_pairs)
pairs['first_letters'] = common_first_letters_cnt(pairs)
pairs['parenthesis'] = parenthesis_feature(df, pairs)
pairs['almost_synonyms'] = almost_synonyms(pairs)
pairs['lcs_feature'] = lcs_feature(pairs)
pairs.to_csv("../dataset/ready_data/potential_pairs_with_features.csv")

6175


In [79]:
pairs.head()

,Unnamed: 0,abbreviation,definition,distance,begin,end,abbreviation_place
0,0,РФ,регуляции репродуктивной функции,14,20,23,36
1,1,РФ,регуляции репродуктивной функции .,13,20,24,36
2,2,РФ,репродуктивной функции,14,21,23,36
3,3,РФ,репродуктивной функции .,13,21,24,36
4,4,РФ,репродуктивной функции . Целью,12,21,25,36


In [102]:
pairs = pd.read_csv("../dataset/ready_data/6_potential_pairs.csv")
df = pd.read_excel("../dataset/ready_data/6_without_stopwords.xlsx")
correct_pairs = pd.read_excel("../dataset/ready_data/6_correct_pairs.xlsx")
pairs['is_it_correct'] = is_it_correct(pairs, correct_pairs)
pairs['first_letters'] = common_first_letters_cnt(pairs)
pairs['parenthesis'] = parenthesis_feature(df, pairs)
pairs['almost_synonyms'] = almost_synonyms(pairs)
pairs['lcs_feature'] = lcs_feature(pairs)
pairs.to_csv("../dataset/ready_data/6_potential_pairs_with_features.csv")

2494


In [101]:
correct_pairs.head()

,ОЩЖ,околощетовидные железы
0,ПГПТ,первичный гиперпаратиреоз
1,ПТГ,паратиреоидный гормон
2,ПТЭ,паратиреоидэктомия


In [92]:
pairs['is_it_correct'] = is_it_correct(pairs, correct_pairs)

6175
l =  0  r =  54  m =  27
l =  27  r =  54  m =  40
l =  40  r =  54  m =  47
l =  40  r =  47  m =  43
l =  40  r =  43  m =  41
l =  41  r =  43  m =  42
done
i =  0
l =  0  r =  54  m =  27
l =  27  r =  54  m =  40
l =  40  r =  54  m =  47
l =  40  r =  47  m =  43
l =  40  r =  43  m =  41
l =  41  r =  43  m =  42
done
i =  1
l =  0  r =  54  m =  27
l =  27  r =  54  m =  40
l =  40  r =  54  m =  47
l =  40  r =  47  m =  43
l =  40  r =  43  m =  41
l =  41  r =  43  m =  42
done
i =  2
l =  0  r =  54  m =  27
l =  27  r =  54  m =  40
l =  40  r =  54  m =  47
l =  40  r =  47  m =  43
l =  40  r =  43  m =  41
l =  41  r =  43  m =  42
done
i =  3
l =  0  r =  54  m =  27
l =  27  r =  54  m =  40
l =  40  r =  54  m =  47
l =  40  r =  47  m =  43
l =  40  r =  43  m =  41
l =  41  r =  43  m =  42
done
i =  4
l =  0  r =  54  m =  27
l =  0  r =  27  m =  13
l =  0  r =  13  m =  6
l =  6  r =  13  m =  9
l =  9  r =  13  m =  11
l =  9  r =  11  m =  10
done
i =  5


i =  56
l =  0  r =  54  m =  27
l =  0  r =  27  m =  13
l =  13  r =  27  m =  20
l =  13  r =  20  m =  16
l =  16  r =  20  m =  18
l =  18  r =  20  m =  19
done
i =  57
l =  0  r =  54  m =  27
l =  0  r =  27  m =  13
l =  13  r =  27  m =  20
l =  13  r =  20  m =  16
l =  16  r =  20  m =  18
l =  18  r =  20  m =  19
done
i =  58
l =  0  r =  54  m =  27
l =  0  r =  27  m =  13
l =  13  r =  27  m =  20
l =  13  r =  20  m =  16
l =  16  r =  20  m =  18
l =  18  r =  20  m =  19
done
i =  59
l =  0  r =  54  m =  27
l =  0  r =  27  m =  13
l =  13  r =  27  m =  20
l =  13  r =  20  m =  16
l =  16  r =  20  m =  18
l =  18  r =  20  m =  19
done
i =  60
l =  0  r =  54  m =  27
l =  0  r =  27  m =  13
l =  13  r =  27  m =  20
l =  13  r =  20  m =  16
l =  16  r =  20  m =  18
l =  18  r =  20  m =  19
done
i =  61
l =  0  r =  54  m =  27
l =  0  r =  27  m =  13
l =  13  r =  27  m =  20
l =  13  r =  20  m =  16
l =  16  r =  20  m =  18
l =  18  r =  20  m =  19
don

i =  109
l =  0  r =  54  m =  27
l =  0  r =  27  m =  13
l =  0  r =  13  m =  6
l =  6  r =  13  m =  9
l =  9  r =  13  m =  11
l =  9  r =  11  m =  10
done
i =  110
l =  0  r =  54  m =  27
l =  0  r =  27  m =  13
l =  0  r =  13  m =  6
l =  6  r =  13  m =  9
l =  9  r =  13  m =  11
l =  9  r =  11  m =  10
done
i =  111
l =  0  r =  54  m =  27
l =  0  r =  27  m =  13
l =  0  r =  13  m =  6
l =  6  r =  13  m =  9
l =  9  r =  13  m =  11
l =  9  r =  11  m =  10
done
i =  112
l =  0  r =  54  m =  27
l =  0  r =  27  m =  13
l =  0  r =  13  m =  6
l =  6  r =  13  m =  9
l =  9  r =  13  m =  11
l =  9  r =  11  m =  10
done
i =  113
l =  0  r =  54  m =  27
l =  0  r =  27  m =  13
l =  0  r =  13  m =  6
l =  6  r =  13  m =  9
l =  9  r =  13  m =  11
l =  9  r =  11  m =  10
done
i =  114
l =  0  r =  54  m =  27
l =  0  r =  27  m =  13
l =  0  r =  13  m =  6
l =  6  r =  13  m =  9
l =  9  r =  13  m =  11
l =  9  r =  11  m =  10
done
i =  115
l =  0  r =  54  m 

i =  160
l =  0  r =  54  m =  27
l =  0  r =  27  m =  13
l =  0  r =  13  m =  6
l =  6  r =  13  m =  9
l =  9  r =  13  m =  11
l =  9  r =  11  m =  10
done
i =  161
l =  0  r =  54  m =  27
l =  0  r =  27  m =  13
l =  0  r =  13  m =  6
l =  6  r =  13  m =  9
l =  9  r =  13  m =  11
l =  9  r =  11  m =  10
done
i =  162
l =  0  r =  54  m =  27
l =  0  r =  27  m =  13
l =  0  r =  13  m =  6
l =  6  r =  13  m =  9
l =  9  r =  13  m =  11
l =  9  r =  11  m =  10
done
i =  163
l =  0  r =  54  m =  27
l =  0  r =  27  m =  13
l =  0  r =  13  m =  6
l =  6  r =  13  m =  9
l =  9  r =  13  m =  11
l =  9  r =  11  m =  10
done
i =  164
l =  0  r =  54  m =  27
l =  0  r =  27  m =  13
l =  0  r =  13  m =  6
l =  6  r =  13  m =  9
l =  9  r =  13  m =  11
l =  9  r =  11  m =  10
done
i =  165
l =  0  r =  54  m =  27
l =  0  r =  27  m =  13
l =  0  r =  13  m =  6
l =  6  r =  13  m =  9
l =  9  r =  13  m =  11
l =  9  r =  11  m =  10
done
i =  166
l =  0  r =  54  m 

l =  27  r =  54  m =  40
l =  40  r =  54  m =  47
l =  40  r =  47  m =  43
l =  43  r =  47  m =  45
l =  45  r =  47  m =  46
done
i =  225
l =  0  r =  54  m =  27
l =  27  r =  54  m =  40
l =  40  r =  54  m =  47
l =  40  r =  47  m =  43
l =  43  r =  47  m =  45
l =  45  r =  47  m =  46
done
i =  226
l =  0  r =  54  m =  27
l =  27  r =  54  m =  40
l =  40  r =  54  m =  47
l =  40  r =  47  m =  43
l =  43  r =  47  m =  45
l =  45  r =  47  m =  46
done
i =  227
l =  0  r =  54  m =  27
l =  27  r =  54  m =  40
l =  40  r =  54  m =  47
l =  40  r =  47  m =  43
l =  43  r =  47  m =  45
l =  45  r =  47  m =  46
done
i =  228
l =  0  r =  54  m =  27
l =  27  r =  54  m =  40
l =  40  r =  54  m =  47
l =  40  r =  47  m =  43
l =  43  r =  47  m =  45
l =  45  r =  47  m =  46
done
i =  229
l =  0  r =  54  m =  27
l =  27  r =  54  m =  40
l =  40  r =  54  m =  47
l =  40  r =  47  m =  43
l =  43  r =  47  m =  45
l =  45  r =  47  m =  46
done
i =  230
l =  0  r =

...

i =  5899
l =  0  r =  54  m =  27
l =  27  r =  54  m =  40
l =  40  r =  54  m =  47
l =  47  r =  54  m =  50
l =  47  r =  50  m =  48
done
i =  5900
l =  0  r =  54  m =  27
l =  27  r =  54  m =  40
l =  40  r =  54  m =  47
l =  47  r =  54  m =  50
l =  47  r =  50  m =  48
done
i =  5901
l =  0  r =  54  m =  27
l =  27  r =  54  m =  40
l =  40  r =  54  m =  47
l =  47  r =  54  m =  50
l =  47  r =  50  m =  48
done
i =  5902
l =  0  r =  54  m =  27
l =  27  r =  54  m =  40
l =  40  r =  54  m =  47
l =  47  r =  54  m =  50
l =  47  r =  50  m =  48
done
i =  5903
l =  0  r =  54  m =  27
l =  27  r =  54  m =  40
l =  40  r =  54  m =  47
l =  47  r =  54  m =  50
l =  47  r =  50  m =  48
done
i =  5904
l =  0  r =  54  m =  27
l =  27  r =  54  m =  40
l =  40  r =  54  m =  47
l =  47  r =  54  m =  50
l =  47  r =  50  m =  48
done
i =  5905
l =  0  r =  54  m =  27
l =  27  r =  54  m =  40
l =  40  r =  54  m =  47
l =  47  r =  54  m =  50
l =  47  r =  50  m =  

l =  21  r =  23  m =  22
done
i =  5982
l =  0  r =  54  m =  27
l =  0  r =  27  m =  13
l =  13  r =  27  m =  20
l =  20  r =  27  m =  23
l =  20  r =  23  m =  21
l =  21  r =  23  m =  22
done
i =  5983
l =  0  r =  54  m =  27
l =  27  r =  54  m =  40
l =  40  r =  54  m =  47
l =  47  r =  54  m =  50
l =  47  r =  50  m =  48
done
i =  5984
l =  0  r =  54  m =  27
l =  27  r =  54  m =  40
l =  40  r =  54  m =  47
l =  47  r =  54  m =  50
l =  47  r =  50  m =  48
done
i =  5985
l =  0  r =  54  m =  27
l =  27  r =  54  m =  40
l =  40  r =  54  m =  47
l =  47  r =  54  m =  50
l =  47  r =  50  m =  48
done
i =  5986
l =  0  r =  54  m =  27
l =  27  r =  54  m =  40
l =  40  r =  54  m =  47
l =  47  r =  54  m =  50
l =  47  r =  50  m =  48
done
i =  5987
l =  0  r =  54  m =  27
l =  27  r =  54  m =  40
l =  40  r =  54  m =  47
l =  47  r =  54  m =  50
l =  47  r =  50  m =  48
done
i =  5988
l =  0  r =  54  m =  27
l =  27  r =  54  m =  40
l =  40  r =  54  m

i =  6034
l =  0  r =  54  m =  27
l =  0  r =  27  m =  13
l =  13  r =  27  m =  20
l =  13  r =  20  m =  16
l =  16  r =  20  m =  18
l =  16  r =  18  m =  17
done
i =  6035
l =  0  r =  54  m =  27
l =  0  r =  27  m =  13
l =  13  r =  27  m =  20
l =  13  r =  20  m =  16
l =  16  r =  20  m =  18
l =  16  r =  18  m =  17
done
i =  6036
l =  0  r =  54  m =  27
l =  0  r =  27  m =  13
l =  13  r =  27  m =  20
l =  13  r =  20  m =  16
l =  16  r =  20  m =  18
l =  16  r =  18  m =  17
done
i =  6037
l =  0  r =  54  m =  27
l =  0  r =  27  m =  13
l =  13  r =  27  m =  20
l =  13  r =  20  m =  16
l =  16  r =  20  m =  18
l =  16  r =  18  m =  17
done
i =  6038
l =  0  r =  54  m =  27
l =  0  r =  27  m =  13
l =  13  r =  27  m =  20
l =  13  r =  20  m =  16
l =  16  r =  20  m =  18
l =  16  r =  18  m =  17
done
i =  6039
l =  0  r =  54  m =  27
l =  0  r =  27  m =  13
l =  13  r =  27  m =  20
l =  13  r =  20  m =  16
l =  16  r =  20  m =  18
l =  16  r =  18 

i =  6088
l =  0  r =  54  m =  27
l =  27  r =  54  m =  40
l =  40  r =  54  m =  47
l =  47  r =  54  m =  50
l =  47  r =  50  m =  48
done
i =  6089
l =  0  r =  54  m =  27
l =  27  r =  54  m =  40
l =  40  r =  54  m =  47
l =  47  r =  54  m =  50
l =  47  r =  50  m =  48
done
i =  6090
l =  0  r =  54  m =  27
l =  27  r =  54  m =  40
l =  40  r =  54  m =  47
l =  47  r =  54  m =  50
l =  47  r =  50  m =  48
done
i =  6091
l =  0  r =  54  m =  27
l =  27  r =  54  m =  40
l =  40  r =  54  m =  47
l =  47  r =  54  m =  50
l =  47  r =  50  m =  48
done
i =  6092
l =  0  r =  54  m =  27
l =  27  r =  54  m =  40
l =  40  r =  54  m =  47
l =  47  r =  54  m =  50
l =  47  r =  50  m =  48
done
i =  6093
l =  0  r =  54  m =  27
l =  27  r =  54  m =  40
l =  40  r =  54  m =  47
l =  47  r =  54  m =  50
l =  47  r =  50  m =  48
done
i =  6094
l =  0  r =  54  m =  27
l =  27  r =  54  m =  40
l =  40  r =  54  m =  47
l =  47  r =  54  m =  50
l =  47  r =  50  m =  

i =  6144
l =  0  r =  54  m =  27
l =  0  r =  27  m =  13
l =  13  r =  27  m =  20
l =  13  r =  20  m =  16
l =  16  r =  20  m =  18
l =  16  r =  18  m =  17
done
i =  6145
l =  0  r =  54  m =  27
l =  0  r =  27  m =  13
l =  13  r =  27  m =  20
l =  13  r =  20  m =  16
l =  16  r =  20  m =  18
l =  16  r =  18  m =  17
done
i =  6146
l =  0  r =  54  m =  27
l =  0  r =  27  m =  13
l =  13  r =  27  m =  20
l =  13  r =  20  m =  16
l =  16  r =  20  m =  18
l =  16  r =  18  m =  17
done
i =  6147
l =  0  r =  54  m =  27
l =  0  r =  27  m =  13
l =  13  r =  27  m =  20
l =  13  r =  20  m =  16
l =  16  r =  20  m =  18
l =  16  r =  18  m =  17
done
i =  6148
l =  0  r =  54  m =  27
l =  0  r =  27  m =  13
l =  13  r =  27  m =  20
l =  13  r =  20  m =  16
l =  16  r =  20  m =  18
l =  16  r =  18  m =  17
done
i =  6149
l =  0  r =  54  m =  27
l =  0  r =  27  m =  13
l =  13  r =  27  m =  20
l =  13  r =  20  m =  16
l =  16  r =  20  m =  18
l =  16  r =  18 

In [86]:
(27+54)//2

40